# QuantumFold-Advantage: Complete Benchmarking Suite

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Tommaso-R-Marena/QuantumFold-Advantage/blob/main/examples/complete_benchmark.ipynb)

**This notebook runs EVERYTHING:**
- Full training pipeline (quantum vs classical)
- Comprehensive evaluation metrics
- Statistical validation with hypothesis testing
- Publication-ready visualizations
- Checkpointing and resume support

**Estimated runtime:** 30-60 minutes with GPU

**Author:** Tommaso R. Marena (The Catholic University of America)

## 1. Setup Environment

In [ ]:
# Check environment
try:
    import google.colab
    IN_COLAB = True
    print('✅ Running in Google Colab')
except ImportError:
    IN_COLAB = False
    print('💻 Running locally')

# Check GPU
import torch
print(f'\n🔥 PyTorch: {torch.__version__}')
print(f'⚡ CUDA: {torch.cuda.is_available()}')

if torch.cuda.is_available():
    print(f'🎮 GPU: {torch.cuda.get_device_name(0)}')
    print(f'💾 Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')
else:
    print('⚠️  No GPU - training will be VERY slow')
    print('   Enable GPU: Runtime > Change runtime type > T4 GPU')
    
    response = input('Continue without GPU? (y/n): ')
    if response.lower() != 'y':
        raise RuntimeError('GPU required for reasonable training time')

In [ ]:
# Mount Google Drive (optional but recommended for saving results)
SAVE_TO_DRIVE = False
DRIVE_PATH = None

if IN_COLAB:
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=False)
        SAVE_TO_DRIVE = True
        DRIVE_PATH = '/content/drive/MyDrive/QuantumFold_Results'
        print('✅ Google Drive mounted successfully!')
        print(f'   Results will be saved to: {DRIVE_PATH}')
        
        # Create directory
        !mkdir -p {DRIVE_PATH}
    except Exception as e:
        print(f'⚠️  Could not mount Google Drive: {e}')
        print('   Results will only be saved locally')
        SAVE_TO_DRIVE = False
else:
    print('💾 Running locally - results saved to ./outputs')

In [ ]:
if IN_COLAB:
    # Clone repository
    print('📦 Cloning repository...')
    !git clone --quiet https://github.com/Tommaso-R-Marena/QuantumFold-Advantage.git 2>/dev/null || true
    %cd QuantumFold-Advantage
    
    # CRITICAL: Install dependencies in correct order to avoid NumPy incompatibility
    print('\n🔧 Installing dependencies (this may take 3-5 minutes)...')
    
    # Step 1: Upgrade pip and clear cache
    !pip install --upgrade --quiet pip setuptools wheel
    !pip cache purge > /dev/null 2>&1 || true
    
    # Step 2: Install NumPy with version constraint FIRST
    print('📊 Installing NumPy (compatible version)...')
    !pip install --quiet --force-reinstall 'numpy>=1.23.0,<2.0.0'
    
    # Step 3: Install SciPy
    !pip install --quiet 'scipy>=1.10.0'
    
    # Step 4: Install autograd (PennyLane dependency)
    print('🔧 Installing quantum computing stack...')
    !pip install --quiet 'autograd>=1.6.2'
    
    # Step 5: Install PennyLane
    !pip install --quiet --no-deps 'pennylane>=0.32.0'
    !pip install --quiet 'autoray>=0.6.11' 'semantic-version>=2.10' 'networkx' 'rustworkx' 'cachetools'
    
    # Step 6: PyTorch (usually pre-installed)
    print('🔥 Verifying PyTorch...')
    !pip install --quiet torch torchvision --index-url https://download.pytorch.org/whl/cu121
    
    # Step 7: Analysis and visualization
    print('📊 Installing analysis tools...')
    !pip install --quiet pandas matplotlib seaborn plotly
    !pip install --quiet scikit-learn 'statsmodels>=0.13'
    !pip install --quiet tqdm pyyaml
    
    # Step 8: Protein tools (optional)
    print('🧬 Installing bioinformatics tools...')
    !pip install --quiet biopython transformers
    # Skip fair-esm if it causes issues
    !pip install --quiet fair-esm 2>/dev/null || echo 'Skipping fair-esm'
    
    print('\n✅ Installation complete!')
    
    # Verify no conflicts
    import warnings
    warnings.filterwarnings('ignore')
    
else:
    print('Skipping installation - running locally')

## 2. Configuration

In [ ]:
import sys
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add to path
if IN_COLAB:
    sys.path.insert(0, '/content/QuantumFold-Advantage')
    os.chdir('/content/QuantumFold-Advantage')
else:
    sys.path.insert(0, str(Path.cwd().parent))

print(f'📍 Working directory: {os.getcwd()}')

# Verify critical imports
print('\n🔍 Verifying installations...')

try:
    import numpy as np
    print(f'✅ NumPy: {np.__version__}')
except ImportError as e:
    print(f'❌ NumPy failed: {e}')

try:
    import pennylane as qml
    print(f'✅ PennyLane: {qml.__version__}')
except ImportError as e:
    print(f'❌ PennyLane failed: {e}')
    print('\n🔧 Attempting to fix...')
    !pip install --force-reinstall --no-cache-dir 'numpy<2.0' 'autograd>=1.6' 'pennylane>=0.32'
    import pennylane as qml
    print(f'✅ PennyLane: {qml.__version__} (after fix)')

# Verify src module
try:
    import src
    print('✅ src module found')
except ImportError:
    print('❌ ERROR: src module not found!')
    print(f'   sys.path: {sys.path[:3]}')
    raise

print('\n🚀 All dependencies verified!')